# Chapter5 오차역전파법

## 5.1 계산 그래프

### 5.1.1 계산 그래프로 풀다
*문제 1 : 현빈 군은 슈퍼에서 1개에 100원인 사과를 2개 샀습니다. 이때 지불 금액을 구하세요. 단 소비세가 10% 부과됩니다.* <hr>
<img style="float: left;" src="equations_and_figures/fig%205-1.png" width="600"> <hr>
<img style="float: left;" src="equations_and_figures/fig%205-2.png" width="600"> <hr>

*문제 2 : 현빈 군은 슈퍼에서 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원입니다. 소비세가 10%일 때 지불 금액을 구하세요.* <hr>
<img style="float: left;" src="equations_and_figures/fig%205-3.png" width="700">

1. 계산 그래프를 구성한다
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다. (순전파 forward propagation)  
오른쪽에서 왼쪽으로 전파 -> 역전파 backward propagation

### 5.1.2 국소적 계산
<img style="float: left;" src="equations_and_figures/fig%205-4.png" width="700">

### 5.1.3 왜 계산 그래프로 푸는가?
<img style="float: left;" src="equations_and_figures/fig%205-5.png" width="600">

##  5.2 연쇄법칙

### 5.2.1 계산 그래프의 역전파
<img style="float: left;" src="equations_and_figures/fig%205-6.png" width="400">

### 5.2.2 연쇄법칙이란?
합성함수  
<img style="float: left;" src="equations_and_figures/e%205.1.png" width="150">

*합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.*  
<img style="float: left;" src="equations_and_figures/e%205.2.png" width="200">
<img style="float: left;" src="equations_and_figures/e%205.3.png" width="140">
<img style="float: left;" src="equations_and_figures/e%205.4.png" width="400">

### 5.2.3 연쇄법칙과 계산 그래프
<img style="float: left;" src="equations_and_figures/fig%205-7.png" width="600">
<img style="float: left;" src="equations_and_figures/fig%205-8.png" width="600">

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파
   $z = x + y$ <br>  
<img style="float: left;" src="equations_and_figures/e%205.5.png" width="100">
<img style="float: left;" src="equations_and_figures/fig%205-9.png" width="600">
<img style="float: left;" src="equations_and_figures/fig%205-10.png" width="600">
<img style="float: left;" src="equations_and_figures/fig%205-11.png" width="600">

### 5.3.2 곱셈 노드의 역전파
$z = xy$ <br>  
<img style="float: left;" src="equations_and_figures/e%205.6.png" width="100">
<img style="float: left;" src="equations_and_figures/fig%205-12.png" width="600">
<img style="float: left;" src="equations_and_figures/fig%205-13.png" width="600">

### 5.3.3 사과 쇼핑의 예
<img style="float: left;" src="equations_and_figures/fig%205-14.png" width="600">

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셉 계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

<img style="float: left;" src="equations_and_figures/fig%205-16.png" width="600">

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("price:", int(price))

price: 220


In [3]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

dApple: 2.2
dApple_num: 110
dTax: 200


### 5.4.2 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

<img style="float: left;" src="equations_and_figures/fig%205-17.png" width="600">

In [ ]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층
<img style="float: left;" src="equations_and_figures/e%205.7.png" width="200">
<img style="float: left;" src="equations_and_figures/e%205.8.png" width="230">
<img style="float: left;" src="equations_and_figures/fig%205-18.png" width="600">

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [ ]:
import numpy as np

x = np.array( [[1.0, -0.5], [-2.0, 3.0]] )
print(x)

In [ ]:
mask = (x <= 0)
print(mask)